## Import

In [1]:
import pandas as pd
import os
import librosa
import numpy as np

## Split into folds

In [2]:
dst = "others/"
processed = 'processed/'
src = 'siren/'
others = os.listdir(processed+dst)
siren = os.listdir(processed+src)
print("No. Others:", len(others))
print("No. Siren:", len(siren))

No. Others: 1957
No. Siren: 1367


In [3]:
def extract_features(filename):
    y, sr = librosa.load(filename, sr=None)
    mfccs = librosa.feature.mfcc(y, sr=sr)
    zero_crossings = librosa.feature.zero_crossing_rate(y, pad=False)
    A = np.vstack([zero_crossings, mfccs])
    return A


In [4]:
fold_size = len(siren)//10
np.random.seed(10)
np.random.shuffle(others)
np.random.shuffle(siren)
fold_list = [{'siren_file':[], 'others_file':[]} for i in range(10)]

In [5]:
#separate list into folds
for i in range(10):
    fold_list[i]['others_file'] = others[i*fold_size:(i+1)*fold_size]
    fold_list[i]['siren_file'] = siren[i*fold_size:(i+1)*fold_size]
        

In [6]:
df_list=[]

In [7]:
for i in range(10):
    curr = {'file':[], 'x':[], 'y':[]}
    for f in fold_list[i]['others_file']:
        curr['file'].append(f)
        curr['x'].append(extract_features(processed+dst+f).astype(np.float32).flatten())
        curr['y'].append(0)
    for f in fold_list[i]['siren_file']:
        curr['file'].append(f)
        curr['x'].append(extract_features(processed+src+f).astype(np.float32).flatten())
        curr['y'].append(1)
    
    df_list.append(pd.DataFrame(curr))


C:\Users\rithi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Pass y=[-0.3033142  -0.3177185  -0.2746277  ...  0.20385742  0.22283936
  0.23760986] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\rithi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Pass y=[ 0.08361816  0.08413696  0.06167603 ... -0.03982544 -0.05331421
 -0.04541016] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\rithi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Pass y=[0.02502441 0.02307129 0.0161438  ... 0.         0.         0.        ] as keyword args. From version 0.10 passing these as pos

## ELM

In [8]:
from tensorflow.keras.utils import to_categorical

In [9]:
from np_elm.elm import ELM, load_model

In [10]:
def merge_folds(test_index):
    df = pd.DataFrame()
    for i in range(10):
        if i != test_index:
            df = df.append(df_list[i])
    df.head()
    x_train, t_train = df['x'], df['y']
    return x_train, t_train

In [11]:
def run_model(test_index):
    # ===============================
    # Load dataset
    # ===============================
    n_classes = 2
    x_train, t_train = merge_folds(test_index)
    x_test, t_test = df_list[test_index]['x'], df_list[test_index]['y']

    # ===============================
    # Preprocess
    # ===============================
    # x_train = x_train.astype(np.float32) / 255.
    # x_train = x_train.reshape(-1, 28**2)
    # x_test = x_test.astype(np.float32) / 255.
    # x_test = x_test.reshape(-1, 28**2)
    x_train = x_train.to_numpy() / 32768.0
    x_train = np.stack(x_train)
    x_test = x_test.to_numpy() / 32768.0
    x_test = np.stack(x_test)
    # x_test = np.stack(x_test.to_numpy())
    t_train = to_categorical(t_train, n_classes).astype(np.float32)
    t_test = to_categorical(t_test, n_classes).astype(np.float32)

    # ===============================
    # Instantiate ELM
    # ===============================
    model = ELM(
        n_input_nodes=x_train[0].shape[0],
        n_hidden_nodes=1000,
        n_output_nodes=n_classes,
        loss='mean_squared_error',
        activation='sigmoid',
        name='elm',
    )

    # ===============================
    # Training
    # ===============================
    model.fit(x_train, t_train)
    train_loss, train_acc = model.evaluate(x_train, t_train, metrics=['loss', 'accuracy'])
    print('train_loss: %f' % train_loss)
    print('train_acc: %f' % train_acc)
    val_loss, val_acc = model.evaluate(x_test, t_test, metrics=['loss', 'accuracy'])
    print('val_loss: %f' % val_loss)
    print('val_acc: %f' % val_acc)

In [12]:
# for i in range(10):
#     run_model(i)

## Random

In [13]:
df = df_list[0]
df_test = df_list[1]
n_classes = 2
x_train, t_train = df['x'], df['y']
x_test, t_test = df_test['x'], df_test['y']
x_train = x_train.to_numpy() / 32768.0
x_train = np.stack(x_train)
x_test = x_test.to_numpy() / 32768.0
x_test = np.stack(x_test)
# x_test = np.stack(x_test.to_numpy())
t_train = to_categorical(t_train, n_classes).astype(np.float32)
t_test = to_categorical(t_test, n_classes).astype(np.float32)
print("xtrain shape", x_train.shape)
print("xtest shape", x_test.shape)
print("t_train shape", t_train.shape)
print("t_test shape", t_test.shape)

xtrain shape (272, 7896)
xtest shape (272, 7896)
t_train shape (272, 2)
t_test shape (272, 2)


In [14]:
# ===============================
# Load dataset
# ===============================
test_index = 3
n_classes = 2
x_train, t_train = merge_folds(test_index)
x_test, t_test = df_list[test_index]['x'], df_list[test_index]['y']

# ===============================
# Preprocess
# ===============================

# x_train = x_train.to_numpy() / 32768.0
# x_train = np.stack(x_train)
x_train = np.stack(x_train.to_numpy())

# x_test = x_test.to_numpy() / 32768.0
# x_test = np.stack(x_test)
x_test = np.stack(x_test.to_numpy())

t_train = to_categorical(t_train, n_classes).astype(np.float32)
t_test = to_categorical(t_test, n_classes).astype(np.float32)

# ===============================
# Instantiate ELM
# ===============================
model = ELM(
    n_input_nodes=x_train[0].shape[0],
    n_hidden_nodes=100,
    n_output_nodes=n_classes,
    loss='mean_squared_error',
    activation='sigmoid',
    name='elm',
)

# ===============================
# Training
# ===============================
model.fit(x_train, t_train)
train_loss, train_acc = model.evaluate(x_train, t_train, metrics=['loss', 'accuracy'])
print('train_loss: %f' % train_loss)
print('train_acc: %f' % train_acc)
val_loss, val_acc = model.evaluate(x_test, t_test, metrics=['loss', 'accuracy'])
print('val_loss: %f' % val_loss)
print('val_acc: %f' % val_acc)

train_loss: 0.063600
train_acc: 0.843954
val_loss: 0.069974
val_acc: 0.812500


c:\Users\rithi\data\ssir\np_elm\elm.py:11: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(-x))


In [15]:
w = model.weights
alpha = w['alpha']
beta = w['beta']
bias = w['bias']
print('alpha', alpha.shape)
print('beta', beta.shape)
print('bias', bias.shape)
print('x_test', x_test.shape)
print('t_test', t_test.shape)

alpha (7896, 100)
beta (100, 2)
bias (100,)
x_test (272, 7896)
t_test (272, 2)


In [16]:
model.predict(x_test[0])

[0.851783096952525, 0.14821690304746363]

In [17]:
print('n_input',model.n_input_nodes)
print('n_hidden',model.n_hidden_nodes)
print('n_output',model.n_output_nodes)
print('out shape', model.output_shape)

n_input 7896
n_hidden 100
n_output 2
out shape (2,)


In [18]:
f = open('x_test.txt', 'w')
f.write(','.join(map(str,x_test.flatten()))+',')
f.close()

In [19]:
f = open('alpha.txt', 'w')
f.write(','.join(map(str,alpha.flatten()))+',')
f.close()
f = open('beta.txt', 'w')
f.write(','.join(map(str,beta.flatten()))+',')
f.close()
f = open('bias.txt', 'w')
f.write(','.join(map(str,bias.flatten()))+',')
f.close()

In [20]:
print(len(str(alpha[0][0]))-2)
print(alpha[7895][99])
print(max(bias.flatten().tolist()))

17
0.49748054293513033
0.0
